In [ ]:
from pathlib import Path
import torch
from gelos.gelosdatamodule import GELOSDataModule
import yaml
from gelos import config
from lightning.pytorch import Trainer
from pathlib import Path
from tqdm import tqdm
from gelos.config import PROJ_ROOT, PROCESSED_DATA_DIR, DATA_VERSION, RAW_DATA_DIR
from terratorch.tasks import EmbeddingGenerationTask


2025-12-04 22:58:37.449 | INFO     | gelos.config:<module>:16 - PROJ_ROOT path is: /app
/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [ ]:
from lightning.pytorch.cli import instantiate_class


In [ ]:
class LenientEmbeddingGenerationTask(EmbeddingGenerationTask):
    def check_file_ids(self, file_ids, x):
        return

def generate_embeddings(yaml_path: Path) -> None:

    with open(yaml_path, "r") as f:
        yaml_config = yaml.safe_load(f)
    
    print(yaml.dump(yaml_config))

    model_name = yaml_config['model']['init_args']['model']
    output_dir = PROCESSED_DATA_DIR / DATA_VERSION / model_name
    output_dir.mkdir(exist_ok=True, parents=True)
    data_root = RAW_DATA_DIR / DATA_VERSION

    # add variables to yaml config so it can be passed to classes
    yaml_config['data']['init_args']['data_root'] = data_root
    yaml_config['model']['init_args']['output_dir'] = output_dir

    gelos_datamodule = GELOSDataModule(**yaml_config['data']['init_args'])
    task = LenientEmbeddingGenerationTask(**yaml_config['model']['init_args'])

    device = 'gpu' if torch.cuda.is_available() else 'cpu'
    trainer = Trainer(accelerator=device, devices=1)

    trainer.predict(model=task, datamodule=gelos_datamodule)


In [13]:

def materialize(config_list):
    if not config_list:
        return None

    def _dummy(**kwargs):
        raise RuntimeError("transform config missing class_path")

    return [
        instantiate_class(item, init=_dummy) if isinstance(item, dict) else item
        for item in config_list
    ]

In [3]:
yaml_config_directory = PROJ_ROOT / 'gelos' / 'configs'
yaml_paths = list(yaml_config_directory.glob('*.yaml'))
print(yaml_paths)

[PosixPath('/app/gelos/configs/terramind_embedding_generation.yaml'), PosixPath('/app/gelos/configs/prithvi_eo_600m_embedding_generation.yaml'), PosixPath('/app/gelos/configs/prithvi_eo_300m_embedding_generation.yaml')]


In [4]:
yaml_path = yaml_paths[1]

## Run Embedding Generation step by step

In [15]:
with open(yaml_path, "r") as f:
        yaml_config = yaml.safe_load(f)

print(yaml.dump(yaml_config))

model_name = yaml_config['model']['init_args']['model']
output_dir = PROCESSED_DATA_DIR / DATA_VERSION / model_name
output_dir.mkdir(exist_ok=True, parents=True)
data_root = RAW_DATA_DIR / DATA_VERSION

# add variables to yaml config so it can be passed to classes
yaml_config['data']['init_args']['data_root'] = data_root
yaml_config['model']['init_args']['output_dir'] = output_dir

# instantiate transform classes if they exist
yaml_config["data"]["init_args"]["transform"] = materialize(
    yaml_config["data"]["init_args"].get("transform")
)
gelos_datamodule = GELOSDataModule(**yaml_config['data']['init_args'])
task = LenientEmbeddingGenerationTask(**yaml_config['model']['init_args'])

device = 'gpu' if torch.cuda.is_available() else 'cpu'
trainer = Trainer(accelerator=device, devices=1)

data:
  class_path: gelos.gelosdatamodule.GELOSDataModule
  init_args:
    bands:
      S2L2A:
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - NIR_BROAD
      - NIR_NARROW
    batch_size: 1
    num_workers: 0
    transform:
    - class_path: ToTensorV2
embedding_extraction_strategies:
  All Patches from April to June:
  - start: 50
    step: 1
    stop: 99
  All Steps of Middle Patch:
  - start: 25
    step: 49
    stop: null
  CLS Token:
  - start: 0
    step: 1
    stop: 1
model:
  class_path: terratorch.tasks.EmbeddingGenerationTask
  init_args:
    embed_file_key: filename
    embedding_pooling: null
    has_cls: true
    model: prithvi_eo_v2_600
    model_args:
      backbone: prithvi_eo_v2_600
      backbone_bands:
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - NIR_BROAD
      - NIR_NARROW
      backbone_pretrained: true
    output_format: parquet
  title: Prithvi EO V2 600M
seed_everything: 0
trainer:
  accelerator: auto
  callbacks: []
  dev

AttributeError: 'function' object has no attribute 'get'

In [17]:
gelos_datamodule.setup(stage = "predict")

In [19]:
vars(gelos_datamodule)

{'_log_hyperparams': False,
 'prepare_data_per_node': True,
 'allow_zero_length_dataloader_with_multiple_devices': False,
 'trainer': None,
 'dataset_class': gelos.gelosdataset.GELOSDataSet,
 'batch_size': 1,
 'num_workers': 0,
 'kwargs': {'predict_transform': [{'class_path': 'torchvision.transforms.v2.Pad',
    'init_args': {'padding': 2, 'padding_mode': 'reflect'}}]},
 'dataset': <gelos.gelosdataset.GELOSDataSet at 0x704a89490910>,
 'train_dataset': None,
 'val_dataset': None,
 'test_dataset': None,
 'predict_dataset': None,
 'train_batch_size': None,
 'val_batch_size': None,
 'test_batch_size': None,
 'predict_batch_size': None,
 'aug': <terratorch.datamodules.generic_pixel_wise_data_module.Normalize at 0x704a89434910>,
 'train_aug': None,
 'val_aug': None,
 'test_aug': None,
 'predict_aug': None,
 'collate_fn': <function terratorch.datamodules.generic_multimodal_data_module.collate_samples(batch_list)>,
 'data_root': PosixPath('/app/data/raw/v0.50.0'),
 'bands': {'S2L2A': ['BLUE',
